In [ ]:
import os
import numpy as np
import pandas as pd
from datetime import datetime

In [ ]:
# Function to load and process files based on the given conditions
def process_files(file_list, column_order, rename_dict, drop_column=None):
    dfs = []
    for file in file_list:
        df = pd.read_csv(file, delimiter=';', on_bad_lines='skip', dtype=str)
        if "Accident_Id" in df.columns:
            df.rename(columns={"Accident_Id": "Num_Acc"}, inplace=True)
        if drop_column and drop_column in df.columns:
            df.drop(columns=[drop_column], inplace=True)
        df = df.reindex(columns=column_order)
        dfs.append(df)
    combined_df = pd.concat(dfs, ignore_index=True)
    combined_df.rename(columns=rename_dict, inplace=True)
    return combined_df

In [ ]:
# Define file paths
source_folder = "source/"
caracteristiques_files = [os.path.join(source_folder, f) for f in os.listdir(source_folder) if f.startswith("caracteristiques")]
lieux_files = [os.path.join(source_folder, f) for f in os.listdir(source_folder) if f.startswith("lieux")]
usagers_files = [os.path.join(source_folder, f) for f in os.listdir(source_folder) if f.startswith("usagers")]
vehicules_files = [os.path.join(source_folder, f) for f in os.listdir(source_folder) if f.startswith("vehicules")]

In [ ]:
# Define column orders and rename dictionaries
caracteristiques_columns = ["Num_Acc", "jour", "mois", "an", "hrmn", "lum", "dep", "com", "agg", "int", "atm", "col", "adr", "lat", "long"]
caracteristiques_rename = {"Num_Acc": "AccID", "jour": "day", "mois": "month", "an": "year", "hrmn": "time", "lum": "lum", 
                           "dep": "dep_code", "com": "com_code", "agg": "location", "int": "int", "atm": "atm_condition", 
                           "col": "collision_type", "adr": "address", "lat": "lat", "long": "long"}

lieux_columns = ["Num_Acc", "catr", "voie", "v1", "v2", "circ", "nbv", "vosp", "prof", "pr", "pr1", "plan", "lartpc", 
                 "larrout", "surf", "infra", "situ", "vma"]
lieux_rename = {"Num_Acc": "AccID", "catr": "route_category", "voie": "route_number", "v1": "route_number_index1", 
                "v2": "alph_route_index", "circ": "traffic_regime", "nbv": "total_number_lanes", "vosp": "reserved_lane_code", 
                "prof": "longitudinal_profile", "pr": "upstream_terminal_number", "pr1": "distance_upstream_terminal", 
                "plan": "plan", "lartpc": "width_central_reservation", "larrout": "width_roadway", "surf": "surface_condition", 
                "infra": "infra", "situ": "accident_situation", "vma": "maximum_speed"}

usagers_columns = ["Num_Acc", "id_vehicule", "num_veh", "place", "catu", "grav", "sexe", "an_nais", "trajet", "secu1", 
                   "secu2", "secu3", "locp", "actp", "etatp"]
usagers_rename = {"Num_Acc": "AccID", "id_vehicule": "vehicleID", "num_veh": "num_veh", "place": "seat", "catu": "user_category", 
                  "grav": "gravity", "sexe": "gender", "an_nais": "birth_year", "trajet": "reason_travel", "secu1": "safety_equipment1", 
                  "secu2": "safety_equipment2", "secu3": "safety_equipment3", "locp": "pedestrian_location", "actp": "pedestrian_action", 
                  "etatp": "pedestrian_involved"}

vehicules_columns = ["Num_Acc", "id_vehicule", "num_veh", "senc", "catv", "obs", "obsm", "choc", "manv", "motor", "occutc"]
vehicules_rename = {"Num_Acc": "AccID", "id_vehicule": "vehicleID", "num_veh": "num_veh", "senc": "traffic_direction", 
                    "catv": "vehicle_category", "obs": "fixed_obstacle", "obsm": "mobile_obstacle", "choc": "initial_impact_point", 
                    "manv": "manv", "motor": "motor", "occutc": "number_occupants_publictransport"}

In [ ]:
# Process each group of files
characteristics = process_files(caracteristiques_files, caracteristiques_columns, caracteristiques_rename)
locations = process_files(lieux_files, lieux_columns, lieux_rename)
users = process_files(usagers_files, usagers_columns, usagers_rename, drop_column="id_usager")
vehicles = process_files(vehicules_files, vehicules_columns, vehicules_rename)

In [ ]:
# Function to replace '#ERREUR' values with NaN and convert to numeric type
def clean_column(column):
    return pd.to_numeric(column.replace('#ERREUR', np.nan), errors='coerce').astype(pd.Int64Dtype())

# Function to check if there are any '#ERREUR' values in each column
def check_error_values(df):
    error_columns = []
    for column in df.columns:
        if df[column].astype(str).str.contains('#ERREUR').any():
            error_columns.append(column)
    return error_columns

In [ ]:
# Get the columns that contain '#ERREUR'
characteristics_with_errors = check_error_values(characteristics)
print(characteristics_with_errors)


In [ ]:
print(characteristics.nunique())
print(characteristics.isna().sum() / len(characteristics))
print(characteristics.info())
print(characteristics.isna().sum())


In [ ]:
# Convert 'lat' and 'long' fields to float in the 'characteristics' dataframe
characteristics['lat'] = characteristics['lat'].str.replace(',', '.').astype(float)
characteristics['long'] = characteristics['long'].str.replace(',', '.').astype(float)

# Convert 'time' field to proper time format in the 'characteristics' dataframe
characteristics['time'] = pd.to_datetime(characteristics['time'], format='%H:%M').dt.time

# Convert 'day', 'month', and 'year' fields to integer in the 'characteristics' dataframe
characteristics['day'] = pd.to_numeric(characteristics['day'], errors='coerce').astype(pd.Int64Dtype())
characteristics['month'] = pd.to_numeric(characteristics['month'], errors='coerce').astype(pd.Int64Dtype())
characteristics['year'] = pd.to_numeric(characteristics['year'], errors='coerce').astype(pd.Int64Dtype())

# Drop fields with a large number of missing values and exclude outliers
characteristics = characteristics.drop(['address'], axis=1)

In [ ]:
locations_with_errors = check_error_values(locations)
print(locations_with_errors)

In [ ]:
print(locations.nunique())
print(locations.isna().sum() / len(locations))
print (locations.info())
print(locations.isna().sum())

In [ ]:
# Replace invalid literals with NaN and convert specified columns in 'locations' dataframe
locations['total_number_lanes'] = pd.to_numeric(locations['total_number_lanes'].replace('#ERREUR', np.nan), errors='coerce').astype(pd.Int64Dtype())
locations['maximum_speed'] = pd.to_numeric(locations['maximum_speed'], errors='coerce').astype(pd.Int64Dtype())
average_number_lanes = int(locations['total_number_lanes'].mean())
locations['total_number_lanes'] = locations['total_number_lanes'].fillna(average_number_lanes).astype(int)
locations['upstream_terminal_number'] = locations['upstream_terminal_number'].replace('(1)', 1)

# Drop fields with a large number of missing values and exclude outliers
locations = locations.drop(['alph_route_index', 'distance_upstream_terminal', 'width_central_reservation', 'width_roadway', 'route_number', 'route_number_index1'], axis=1)


In [ ]:
# Insert the code
IQR_maximum_speed = locations["maximum_speed"].quantile(0.75)-locations["maximum_speed"].quantile(0.25)
# Lower bound
I1_maximum_speed = locations["maximum_speed"].quantile(0.25) - 1.5 * IQR_maximum_speed

# Upper bound
I2_maximum_speed = locations["maximum_speed"].quantile(0.75) + 1.5 * IQR_maximum_speed
print()
print("IQR :", IQR_maximum_speed, end="\n\n")
print("Range :[", I1_maximum_speed, ";", I2_maximum_speed, "]")
locations.loc[(locations['maximum_speed'] < -1) | (locations['maximum_speed'] >125), 'maximum_speed']

In [ ]:
vehicles_with_errors = check_error_values(vehicles)
print(vehicles_with_errors)

In [ ]:
print(vehicles.nunique())
print(vehicles.isna().sum() / len(vehicles))
print (vehicles.info())
print(vehicles.isna().sum())

In [ ]:
# Drop fields with a large number of missing values and exclude outliers
vehicles = vehicles.drop(['number_occupants_publictransport'], axis=1)


In [ ]:
users_with_errors = check_error_values(users)
print(users_with_errors)

In [ ]:
print(users.nunique())
print(users.isna().sum() / len(users))
print (users.info())
print(users.isna().sum())

In [ ]:
# Replace invalid literals with NaN and convert specified columns in 'users' dataframe
#users['birth_year'] = users['birth_year'].replace('#ERREUR', np.nan)
users['birth_year'] = pd.to_numeric(users['birth_year'], errors='coerce')
average_birth_year = int(users['birth_year'].mean())
users['birth_year'] = users['birth_year'].fillna(average_birth_year).astype(int)

#Calculate 'age'
current_year = datetime.now().year
users['age'] = current_year - users['birth_year']

In [ ]:
# Insert the code
IQR = users["age"].quantile(0.75)-users["age"].quantile(0.25)
# Lower bound
I1 = users["age"].quantile(0.25) - 1.5 * IQR

# Upper bound
I2 = users["age"].quantile(0.75) + 1.5 * IQR
print()
print("IQR :", IQR, end="\n\n")
print("Range :[", I1, ";", I2, "]")

users.loc[(users['age'] < 0) | (users['age'] > 97), 'age']

In [ ]:
print((characteristics == " -1").sum() / len(users))
print((locations == " -1").sum() / len(users))
print((users == " -1").sum() / len(users))
print((vehicles == " -1").sum() / len(users))


In [ ]:
# Drop fields with a large number of missing values and exclude outliers
characteristics = characteristics[characteristics['lum'] != ' -1']
characteristics = characteristics[characteristics['int'] != ' -1']
characteristics = characteristics[characteristics['atm_condition'] != ' -1']
characteristics = characteristics[characteristics['collision_type'] != ' -1']


locations = locations[locations['plan'] != ' -1']
locations = locations[locations['infra'] != ' -1']
locations = locations[locations['traffic_regime'] != ' -1']
locations = locations[locations['reserved_lane_code'] != ' -1']
locations = locations[locations['surface_condition'] != ' -1']
locations = locations[locations['accident_situation'] != ' -1']
locations = locations[locations['longitudinal_profile'] != ' -1']
locations = locations[(locations['maximum_speed'] >= 5) & (locations['maximum_speed'] <= 125)]

vehicles = vehicles[vehicles['motor'] != ' -1']
vehicles = vehicles[vehicles['manv'] != ' -1']
vehicles = vehicles[vehicles['fixed_obstacle'] != ' -1']
vehicles = vehicles[vehicles['mobile_obstacle'] != ' -1']
vehicles = vehicles[vehicles['vehicle_category'] != ' -1']
vehicles = vehicles[vehicles['traffic_direction'] != ' -1']
vehicles = vehicles[vehicles['initial_impact_point'] != ' -1']

users = users.drop(['safety_equipment2', 'safety_equipment3', 'pedestrian_location', 'pedestrian_action', 'pedestrian_involved'], axis=1)
users = users[users['seat'] != ' -1']
users = users[users['gender'] != ' -1']
users = users[users['gravity'] != ' -1']
users = users[users['reason_travel'] != ' -1']
users = users[users['safety_equipment1'] != ' -1']
users = users[(users['age'] >= 0) & (users['age'] <= 97)]

<font size="6">  
    Merge Dataframes
</font> 

In [ ]:
# Merge dataframes on AccID
data = characteristics.merge(locations, on='AccID').merge(users, on='AccID').merge(vehicles, on='AccID')

In [ ]:
# Delete duplicate rows
data = data.drop_duplicates()

In [ ]:
data.info()

In [ ]:
pd.set_option('display.max_columns', None)
data.head(50)

<font size="6">  
    Export Dataframes
</font>  

In [ ]:
data.to_csv('data.csv', index=False)